In [0]:
from pyspark.sql.functions import col, explode_outer
from pyspark.sql.functions import struct


In [0]:
df = spark.read.json('s3://data-pismo/extraction/*.json',multiLine=True)

In [0]:
df.dtypes

Out[93]: [('end_date', 'string'),
 ('privacy', 'string'),
 ('quotes',
  'struct<2023-09-18:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-19:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-20:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-21:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-22:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-23:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-24:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-25:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-26:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-27:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-28:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-29:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-09-30:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-10-01:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-10-02:struct<USDBRL:double,USDCLP:double,USDEUR:double>,2023-

In [0]:
df.show()

+----------+--------------------+--------------------+------+----------+-------+--------------------+---------+
|  end_date|             privacy|              quotes|source|start_date|success|               terms|timeframe|
+----------+--------------------+--------------------+------+----------+-------+--------------------+---------+
|2023-10-18|https://currencyl...|{{4.855203, 886.6...|   USD|2023-09-18|   true|https://currencyl...|     true|
+----------+--------------------+--------------------+------+----------+-------+--------------------+---------+



In [0]:
df_treatment = df.unpivot('source', ['quotes.*'],  'var',  'value')

In [0]:
df_treatment.show()

+------+----------+--------------------+
|source|       var|               value|
+------+----------+--------------------+
|   USD|2023-09-18|{4.855203, 886.61...|
|   USD|2023-09-19|{4.867301, 884.35...|
|   USD|2023-09-20|{4.880297, 882.59...|
|   USD|2023-09-21|{4.9364, 889.7198...|
|   USD|2023-09-22|{4.935904, 894.20...|
|   USD|2023-09-23|{4.919286, 885.28...|
|   USD|2023-09-24|{4.935404, 894.84...|
|   USD|2023-09-25|{4.969802, 903.40...|
|   USD|2023-09-26|{4.987696, 904.09...|
|   USD|2023-09-27|{5.043697, 909.12...|
|   USD|2023-09-28|{5.034296, 905.39...|
|   USD|2023-09-29|{5.032904, 891.15...|
|   USD|2023-09-30|{5.044254, 901.48...|
|   USD|2023-10-01|{5.044298, 891.15...|
|   USD|2023-10-02|{5.06303, 908.580...|
|   USD|2023-10-03|{5.167798, 917.59...|
|   USD|2023-10-04|{5.156103, 915.33...|
|   USD|2023-10-05|{5.1665, 914.4001...|
|   USD|2023-10-06|{5.150104, 909.69...|
|   USD|2023-10-07|{5.143881, 909.69...|
+------+----------+--------------------+
only showing top

In [0]:
df_final = df_treatment.select('source', 'var', 'value.*')

In [0]:
df_final.show()

+------+----------+--------+----------+--------+
|source|       var|  USDBRL|    USDCLP|  USDEUR|
+------+----------+--------+----------+--------+
|   USD|2023-09-18|4.855203|886.619723|0.935255|
|   USD|2023-09-19|4.867301|884.350185| 0.93619|
|   USD|2023-09-20|4.880297|882.598241| 0.93885|
|   USD|2023-09-21|  4.9364|889.719802| 0.93799|
|   USD|2023-09-22|4.935904|894.203912| 0.93705|
|   USD|2023-09-23|4.919286|885.285989| 0.93705|
|   USD|2023-09-24|4.935404|894.840397| 0.93915|
|   USD|2023-09-25|4.969802|903.409698|0.944165|
|   USD|2023-09-26|4.987696|904.098478|0.946145|
|   USD|2023-09-27|5.043697|909.129666| 0.95194|
|   USD|2023-09-28|5.034296|905.399662| 0.94677|
|   USD|2023-09-29|5.032904|891.150396|0.944604|
|   USD|2023-09-30|5.044254|901.487744|0.944604|
|   USD|2023-10-01|5.044298|891.150207| 0.94665|
|   USD|2023-10-02| 5.06303|908.580165| 0.95417|
|   USD|2023-10-03|5.167798|917.599964| 0.95505|
|   USD|2023-10-04|5.156103|915.330083| 0.95178|
|   USD|2023-10-05| 

In [0]:
df_final.write.mode('overwrite').format('csv').save("s3://data-pismo/cleaned/teste_pismo.csv", header=True)